In [104]:
from elasticsearch import Elasticsearch
import random

es = Elasticsearch([{'host':'localhost','port':9200}])
es

indx = 'redditors.followers.sample'

try:
    es.indices.create(index = indx)
except:
    pass

In [108]:
results = es.search(scroll='360m', body =       
{
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "must": [
            {
              "bool": {
                "must_not": [
                  {
                    "exists": {
                      "field": "commenters_sampled"
                    }
                  }
                ]
              }
            },
            {
              "exists": {
                "field": "commenters"
              }
            }
          ]
        }
      },
      "random_score": {
        "seed": random.randint(1, 1000)
      },
      "boost_mode": "replace"
    }
  }
}, index = 'redditors', size = 10000)

In [109]:
for result in results['hits']['hits']:
    commenters = result['_source']['commenters']
    if len(commenters) >= 100 and len(commenters) <= 160:
        for commenter in commenters:
            doc = {}
            doc['name'] = commenter
            es.index(index = indx, id = commenter, body = doc)

        update = {
            "doc": {
                  "commenters_sampled": True
            }
        }
        es.update(index = 'redditors', id = result['_id'], body = update)